In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from src.data_handling.mmg_detection_datasets import *
from src.visualizations.plot_image import plot_image_opencv_fit_window
from src.data_augmentation.breast_density.data.resize_image import *
import torch

from src.preprocessing.histogram_standardization import get_hist_stand_landmarks, apply_hist_stand_landmarks



pathologies = None #['mass', 'calcifications', 'suspicious_calcifications', 'architectural_distortion'] # None to select all
status = ['Normal', 'Benign', 'Malignant'] #['Normal'] 
# Resize images keeping aspect ratio
ROOT_DATA_PATH = '/home/akis-linardos/Datasets'
# Call to the OPTIMAM Dataloader
csv_path = '/home/akis-linardos/Datasets/CMMD/raw_metadata.csv'
other_csv = '/home/akis-linardos/Datasets/CMMD/CMMD_clinicaldata_revision.xlsx'
dataset_path = '/home/akis-linardos/Datasets/CMMD/data'

In [2]:
md_csv=pd.read_csv(csv_path)

In [3]:
md_csv['Subject ID']

0       D1-0003
1       D1-0001
2       D1-0005
3       D1-0004
4       D1-0002
         ...   
1769    D2-0745
1770    D2-0746
1771    D2-0747
1772    D2-0749
1773    D2-0748
Name: Subject ID, Length: 1774, dtype: object

In [4]:
# md_csv.groupby(['Study UID']).count()

In [5]:
# md_csv.groupby(['Series UID']).count()

In [6]:
md_csv

,Series UID,Collection,3rd Party Analysis,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp
0,1.3.6.1.4.1.14519.5.2.1.1239.1759.292560899611...,CMMD,NO,https://doi.org/10.7937/tcia.eqde4b16,D1-0003,1.3.6.1.4.1.14519.5.2.1.1239.1759.113089024322...,NaN,07-18-2011,NaN,NaN,MG,Digital Mammography X-Ray Image Storage - For ...,1.2.840.10008.5.1.4.1.1.1.2,2,8.79 MB,./CMMD/D1-0003/07-18-2011-NA-NA-25491/1.000000...,2022-02-22T14:26:56.454
1,1.3.6.1.4.1.14519.5.2.1.1239.1759.623006463861...,CMMD,NO,https://doi.org/10.7937/tcia.eqde4b16,D1-0001,1.3.6.1.4.1.14519.5.2.1.1239.1759.335790956129...,NaN,07-18-2010,NaN,NaN,MG,Digital Mammography X-Ray Image Storage - For ...,1.2.840.10008.5.1.4.1.1.1.2,2,8.79 MB,./CMMD/D1-0001/07-18-2010-NA-NA-79377/1.000000...,2022-02-22T14:26:57.31
2,1.3.6.1.4.1.14519.5.2.1.1239.1759.411712488433...,CMMD,NO,https://doi.org/10.7937/tcia.eqde4b16,D1-0005,1.3.6.1.4.1.14519.5.2.1.1239.1759.249693111563...,NaN,07-18-2011,NaN,NaN,MG,Digital Mammography X-Ray Image Storage - For ...,1.2.840.10008.5.1.4.1.1.1.2,2,8.79 MB,./CMMD/D1-0005/07-18-2011-NA-NA-52347/1.000000...,2022-02-22T14:27:01.533
3,1.3.6.1.4.1.14519.5.2.1.1239.1759.328825651506...,CMMD,NO,https://doi.org/10.7937/tcia.eqde4b16,D1-0004,1.3.6.1.4.1.14519.5.2.1.1239.1759.132173027545...,NaN,07-18-2011,NaN,NaN,MG,Digital Mammography X-Ray Image Storage - For ...,1.2.840.10008.5.1.4.1.1.1.2,2,8.79 MB,./CMMD/D1-0004/07-18-2011-NA-NA-14914/1.000000...,2022-02-22T14:27:02.092
4,1.3.6.1.4.1.14519.5.2.1.1239.1759.610823649257...,CMMD,NO,https://doi.org/10.7937/tcia.eqde4b16,D1-0002,1.3.6.1.4.1.14519.5.2.1.1239.1759.241519791051...,NaN,07-18-2010,NaN,NaN,MG,Digital Mammography X-Ray Image Storage - For ...,1.2.840.10008.5.1.4.1.1.1.2,2,8.79 MB,./CMMD/D1-0002/07-18-2010-NA-NA-49231/1.000000...,2022-02-22T14:27:06.867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1769,1.3.6.1.4.1.14519.5.2.1.1239.1759.254393425808...,CMMD,NO,https://doi.org/10.7937/tcia.eqde4b16,D2-0745,1.3.6.1.4.1.14519.5.2.1.1239.1759.829075352788...,NaN,07-18-2011,NaN,NaN,MG,Digital Mammography X-Ray Image Storage - For ...,1.2.840.10008.5.1.4.1.1.1.2,4,17.57 MB,./CMMD/D2-0745/07-18-2011-NA-NA-87264/1.000000...,2022-02-22T15:51:13.357
1770,1.3.6.1.4.1.14519.5.2.1.1239.1759.972296336858...,CMMD,NO,https://doi.org/10.7937/tcia.eqde4b16,D2-0746,1.3.6.1.4.1.14519.5.2.1.1239.1759.291758813596...,NaN,07-18-2011,NaN,NaN,MG,Digital Mammography X-Ray Image Storage - For ...,1.2.840.10008.5.1.4.1.1.1.2,4,17.57 MB,./CMMD/D2-0746/07-18-2011-NA-NA-61307/1.000000...,2022-02-22T15:51:18.447
1771,1.3.6.1.4.1.14519.5.2.1.1239.1759.240741173543...,CMMD,NO,https://doi.org/10.7937/tcia.eqde4b16,D2-0747,1.3.6.1.4.1.14519.5.2.1.1239.1759.240317944400...,NaN,07-18-2011,NaN,NaN,MG,Digital Mammography X-Ray Image Storage - For ...,1.2.840.10008.5.1.4.1.1.1.2,4,17.57 MB,./CMMD/D2-0747/07-18-2011-NA-NA-07955/1.000000...,2022-02-22T15:51:25.041
1772,1.3.6.1.4.1.14519.5.2.1.1239.1759.311460119741...,CMMD,NO,https://doi.org/10.7937/tcia.eqde4b16,D2-0749,1.3.6.1.4.1.14519.5.2.1.1239.1759.134277145908...,NaN,07-17-2012,NaN,NaN,MG,Digital Mammography X-Ray Image Storage - For ...,1.2.840.10008.5.1.4.1.1.1.2,4,17.57 MB,./CMMD/D2-0749/07-17-2012-NA-NA-85032/1.000000...,2022-02-22T15:51:25.901


In [7]:
md_csv['Number of Images'].sum()

5198

In [8]:
csv = pd.read_excel(other_csv)

In [9]:
csv

,ID1,LeftRight,Age,number,abnormality,classification,subtype
0,D1-0001,R,44,2,calcification,Benign,NaN
1,D1-0002,L,40,2,calcification,Benign,NaN
2,D1-0003,L,39,2,calcification,Benign,NaN
3,D1-0004,L,41,2,calcification,Benign,NaN
4,D1-0005,R,42,2,calcification,Benign,NaN
...,...,...,...,...,...,...,...
1867,D2-0745,L,43,2,mass,Malignant,triple negative
1868,D2-0746,R,59,2,mass,Malignant,HER2-enriched
1869,D2-0747,R,53,2,mass,Malignant,Luminal B
1870,D2-0748,R,29,2,mass,Malignant,Luminal B


In [10]:
md_csv.columns

Index(['Series UID', 'Collection', '3rd Party Analysis',
       'Data Description URI', 'Subject ID', 'Study UID', 'Study Description',
       'Study Date', 'Series Description', 'Manufacturer', 'Modality',
       'SOP Class Name', 'SOP Class UID', 'Number of Images', 'File Size',
       'File Location', 'Download Timestamp'],
      dtype='object')

In [11]:
csv.columns

Index(['ID1', 'LeftRight', 'Age', 'number', 'abnormality', 'classification',
       'subtype'],
      dtype='object')

In [12]:
D1sample = os.path.join(csv['ID1'][0],'*','*')

In [13]:
csv['ID1']

0       D1-0001
1       D1-0002
2       D1-0003
3       D1-0004
4       D1-0005
         ...   
1867    D2-0745
1868    D2-0746
1869    D2-0747
1870    D2-0748
1871    D2-0749
Name: ID1, Length: 1872, dtype: object

# append 'status' column to md_csv. Make a list that has all labels matched to 'Subject ID' from md_csv. Match label to ID1 to Subject ID

In [14]:
l = [None]*1774

In [15]:
md_csv['status']=l
md_csv['pathology']=l
md_csv['laterality']=l

In [16]:
for i, ID in enumerate(md_csv['Subject ID']):
    # print(csv[csv['ID1']==ID]['classification'].values)
    md_csv['status'][md_csv['Subject ID']==ID]=csv[csv['ID1']==ID]['classification'].values[0]
    md_csv['pathology'][md_csv['Subject ID']==ID]=csv[csv['ID1']==ID]['abnormality'].values[0]
    md_csv['laterality'][md_csv['Subject ID']==ID]=csv[csv['ID1']==ID]['LeftRight'].values[0]

    # if i >10:
        # break
# for i in csv['ID1']
# md_csv['status']

/home/akis-linardos/BFP/docker/fl_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/akis-linardos/BFP/docker/fl_env/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/akis-linardos/BFP/docker/fl_env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

# Clean up the path names

In [17]:
weird=[]

In [18]:
for i, ID in enumerate(md_csv['Subject ID']):
    vv=csv[csv['ID1']==ID]['classification'].values
    if len(vv)>1:
        # print(csv[csv['ID1']==ID]['classification'].values)
        weird.append(md_csv['Subject ID'][md_csv['Subject ID']==ID])

In [19]:
md_csv['File Location']=md_csv['File Location'].replace({'./C': 'C'}, regex=True)

In [20]:
md_csv.columns

Index(['Series UID', 'Collection', '3rd Party Analysis',
       'Data Description URI', 'Subject ID', 'Study UID', 'Study Description',
       'Study Date', 'Series Description', 'Manufacturer', 'Modality',
       'SOP Class Name', 'SOP Class UID', 'Number of Images', 'File Size',
       'File Location', 'Download Timestamp', 'status', 'pathology',
       'laterality'],
      dtype='object')

# Append new rows depending on number of images

In [26]:
newDF=pd.DataFrame(columns=md_csv.columns)
for i, ID in enumerate((md_csv['Subject ID'])):
    dicoms = os.listdir(os.path.join(ROOT_DATA_PATH, md_csv['File Location'][i]))
    for i, dicom in enumerate(dicoms):
        row_of_interest=md_csv[md_csv['Subject ID']==ID]
        row_of_interest['File Location']=row_of_interest['File Location']+'/'+dicom
        newDF=newDF.append(row_of_interest, ignore_index=True)
    # newDF.append([md_csv[md_csv['Subject ID']==ID]]*len(dicoms),ignore_index=True)
    # break

/home/akis-linardos/BFP/docker/fl_env/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
newDF['File Location']

0       CMMD/D1-0003/07-18-2011-NA-NA-25491/1.000000-N...
1       CMMD/D1-0003/07-18-2011-NA-NA-25491/1.000000-N...
2       CMMD/D1-0001/07-18-2010-NA-NA-79377/1.000000-N...
3       CMMD/D1-0001/07-18-2010-NA-NA-79377/1.000000-N...
4       CMMD/D1-0005/07-18-2011-NA-NA-52347/1.000000-N...
                              ...                        
5193    CMMD/D2-0749/07-17-2012-NA-NA-85032/1.000000-N...
5194    CMMD/D2-0748/07-18-2011-NA-NA-50416/1.000000-N...
5195    CMMD/D2-0748/07-18-2011-NA-NA-50416/1.000000-N...
5196    CMMD/D2-0748/07-18-2011-NA-NA-50416/1.000000-N...
5197    CMMD/D2-0748/07-18-2011-NA-NA-50416/1.000000-N...
Name: File Location, Length: 5198, dtype: object

In [23]:
newDF.columns

Index(['Series UID', 'Collection', '3rd Party Analysis',
       'Data Description URI', 'Subject ID', 'Study UID', 'Study Description',
       'Study Date', 'Series Description', 'Manufacturer', 'Modality',
       'SOP Class Name', 'SOP Class UID', 'Number of Images', 'File Size',
       'File Location', 'Download Timestamp', 'status', 'pathology',
       'laterality'],
      dtype='object')

In [24]:
df_to_save = newDF
df_to_save = df_to_save.rename(columns={'Subject ID':'client_id', 'Study UID':'study_id', 'Series UID':'series_id', 'File Location':'image_filepath', "Manufacturer":"manufacturer"})

In [29]:
df_to_save['view']=['CC']*len(df_to_save)
df_to_save['image_id'] = range(1,len(df_to_save)+1)
df_to_save['view'][df_to_save['image_id']%2==0]='MLO'

/home/akis-linardos/BFP/docker/fl_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
df_to_save.to_csv("/home/akis-linardos/Datasets/CMMD/info.csv")

# EVEN NUMBERS MLO. DO IT

# JUNK BELOW

In [46]:
newDF=newDF.append(md_csv[md_csv['Subject ID']==ID])

In [66]:
csv

,ID1,LeftRight,Age,number,abnormality,classification,subtype
0,D1-0001,R,44,2,calcification,Benign,NaN
1,D1-0002,L,40,2,calcification,Benign,NaN
2,D1-0003,L,39,2,calcification,Benign,NaN
3,D1-0004,L,41,2,calcification,Benign,NaN
4,D1-0005,R,42,2,calcification,Benign,NaN
...,...,...,...,...,...,...,...
1867,D2-0745,L,43,2,mass,Malignant,triple negative
1868,D2-0746,R,59,2,mass,Malignant,HER2-enriched
1869,D2-0747,R,53,2,mass,Malignant,Luminal B
1870,D2-0748,R,29,2,mass,Malignant,Luminal B


In [46]:
inbreast_csv = pd.read_csv('/home/lidia-garrucho/datasets/INBREAST/INbreast_updated_cropped_breast.csv')
inbreast_csv

,patient_id,laterality,view,acquisition_date,image_id,ACR,BIRADS,scan_path,calcification_mask,mass_mask,...,mass_0_y1,mass_0_y2,mass_1_x1,mass_1_x2,mass_1_y1,mass_1_y2,mass_2_x1,mass_2_x2,mass_2_y1,mass_2_y2
0,61b13c59bcba149e,R,CC,201001,22678622,4,1,61b13c59bcba149e/RIGHT_CC/scan_0.nii.gz,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,61b13c59bcba149e,L,CC,201001,22678646,4,3,61b13c59bcba149e/LEFT_CC/scan_0.nii.gz,NaN,61b13c59bcba149e/LEFT_CC/s0_mask_mass.nii.gz,...,827.0,1153.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,61b13c59bcba149e,R,MLO,201001,22678670,4,1,61b13c59bcba149e/RIGHT_MLO/scan_0.nii.gz,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,61b13c59bcba149e,L,MLO,201001,22678694,4,3,61b13c59bcba149e/LEFT_MLO/scan_0.nii.gz,NaN,61b13c59bcba149e/LEFT_MLO/s0_mask_mass.nii.gz,...,2209.0,2458.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6bd24a0a42c19ce1,R,CC,201001,22614074,2,5,6bd24a0a42c19ce1/RIGHT_CC/scan_0.nii.gz,6bd24a0a42c19ce1/RIGHT_CC/s0_mask_calc.nii.gz,6bd24a0a42c19ce1/RIGHT_CC/s0_mask_mass.nii.gz,...,1479.0,1883.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,45c7f44839fd9e68,L,MLO,201001,22613848,2,1,45c7f44839fd9e68/LEFT_MLO/scan_0.nii.gz,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
406,f0f1a133837b5137,L,MLO,200802,24055725,4,2,f0f1a133837b5137/LEFT_MLO/scan_0.nii.gz,f0f1a133837b5137/LEFT_MLO/s0_mask_calc.nii.gz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407,f0f1a133837b5137,R,MLO,200802,24055752,4,2,f0f1a133837b5137/RIGHT_MLO/scan_0.nii.gz,f0f1a133837b5137/RIGHT_MLO/s0_mask_calc.nii.gz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,f0f1a133837b5137,L,CC,200802,24055779,4,2,f0f1a133837b5137/LEFT_CC/scan_0.nii.gz,f0f1a133837b5137/LEFT_CC/s0_mask_calc.nii.gz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# optimam_csv=pd.read_csv('/home/lidia-garrucho/datasets/OPTIMAM/png_screening_cropped_fixed/client_images_screening.csv')
# optimam_csv

In [36]:
sorted_md = md_csv['Subject ID'].sort_values().tolist()

In [40]:
for val in csv['ID1']:
    if val not in sorted_md:
        print(val)

D2-0503


In [47]:
csv['ID1'][csv['ID1'] == 'D2-0503']

1620    D2-0503
Name: ID1, dtype: object